__Linear Regression with TensorFlow__
There are three ways to fit linear regression model to the data

1. Normal Equation
2. Maximum Likelihood Estimation
3. Stochastic Gradient Descent with Mini-Batches 

In [7]:
import os
import numpy as np
import math as m

import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import  Axes3D

In [8]:
import pandas as pd
import tensorflow as tf

In [9]:
#utility function to make this notebook's output stable across the runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

__We use artificial data for the following regression__<br>
y(x)=a+$b_1$.$X_1$+$b_2$.$X_2$+$b_3$.$X_3$+$\sigma$.$\epsilon$ <br>
where $\epsilon$ ~ N(0,1) as a Guassian noise and $\sigma$ is its volatility, with the following choice of parameters:<br>
a=1.0<br>
$b_1$,$b_2$, $b_3$ = (0.5, 0.2, 0.1)<br>
$\sigma$=0.1<br>
$X_1$, $X_2$, $X_3$ will be uniformly distributed in [-1,1]

__Generate data__

In [12]:
n_points=5000
n_features=3

bias=np.ones(n_points).reshape(-1,1)
low=-np.ones((n_points, n_features),'float')
high=np.ones((n_points, n_features),'float')

#simulated features/inputs 
X=np.random.uniform(low=low, high=high)

#simulated noise
noise=np.random.normal(size=(n_points,1))

#outputs
weights=np.array([1.0,0.5,0.2,0.1])  #here a=0.1
noise_std=0.1
Y=weights[0]*bias+np.dot(X,weights[1:]).reshape(-1,1)+noise_std*noise

#split to the train and test set
train_test_split=4 #1/4 of the data is used for test

n_test=int(n_points/train_test_split)
n_train=n_points-n_test

X_train= X[:n_train,:]
Y_train=Y[:n_train,:].reshape(-1,1)

X_test=X[n_train:,:]
Y_test=Y[n_train:,:].reshape(-1,1)


In [20]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((3750, 3), (3750, 1), (1250, 3), (1250, 1))

__Linear Regression with NumPy__

In [25]:
#add the column of ones
X=np.hstack((np.ones(n_train).reshape(-1,1),X_train))

theta_numpy=np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y_train) 

print(theta_numpy)

[[0.99753201]
 [0.50184089]
 [0.20008112]
 [0.09929622]]


__Linear Regression with sklearn__

In [31]:
from sklearn.linear_model import LinearRegression

lin_reg=LinearRegression()
lin_reg.fit(X_train,Y_train)

#lin_reg.intercept_ is the bias term, lin_reg.coef_ is the weights corresponding to given features

print(np.r_[lin_reg.intercept_.reshape(-1,1),lin_reg.coef_.T]) #np.r_ joins the array vertically like np.vstack

[[0.99753201]
 [0.50184089]
 [0.20008112]
 [0.09929622]]


__Linear Regression with Tensorflow__

In [32]:
#add the column of ones like we added in the Numpy method
X_np=np.hstack((np.ones(n_train).reshape(-1,1),X_train))

X=tf.constant(X_np,dtype=tf.float32, name='X')
y=tf.constant(Y_train, dtype=tf.float32, name='y')
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_val=theta.eval()
    
theta_val

#This is same as Numpy method. Just that we used the Tensorflow functions instead Numpy function for Matrix calculations

array([[0.9975319 ],
       [0.5018408 ],
       [0.20008114],
       [0.09929623]], dtype=float32)

__Create class for Linear Regression__

In [27]:
class LinRegressNormalEq:
    def __init__(self, n_features, learning_rate=0.5, L=0):
        import math as m
        #input placeholders
        self.X=tf.placeholder(tf.float32,[None,n_features],name='X')
        self.Y=tf.placeholder(tf.float32,[None,1],name='Y')
        
        #regression parameters for the analytical solution using Normal Equation
        self.theta_in=tf.placeholder(tf.float32,[n_features+1,None])
        
        #Augmented data matrix is obtained by adding a column of ones to the data matrix
        data_plus_bias=tf.concat([tf.ones([tf.shape(self.X)[0],1]),self.X],axis=1)
        XT=tf.transpose(data_plus_bias)
        
        # The normal equation for Linear Regression
        self.theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, data_plus_bias)), XT), self.Y)
        
        # mean square error in terms of theta = theta_in
        self.lr_mse = tf.reduce_mean(tf.square(tf.matmul(data_plus_bias, self.theta_in) - self.Y))
        
    #enter() and exit() function needs to be created if class is used with 'with' block
    def __enter__(self):
        print("__enter__")
        return self
    
    def __exit__(self, type, value, traceback): #__exit__ function has 4 arguments by default
        print("__exit__")
    

In [32]:
def run_normal_eq(X_train, Y_train, X_test, Y_test, learning_rate=0.05):
    """
    Implements normal equation using tensorflow, trains the model using training data set
    Tests the model quality by computing mean square error (MSE) of the test data set
    
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    X_test  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_test - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    
    Return a tuple of:
        - np.array of size (k+1 by 1) of regression coefficients
        - mean square error (MSE) of the test data set
        - mean square error (MSE) of the training data set
    """
    # create an instance of the Linear Regression model class  
    n_features = X_train.shape[1]
    #model = LinRegressNormalEq(n_features=n_features, learning_rate=learning_rate)
    
    with LinRegressNormalEq(n_features=n_features, learning_rate=learning_rate) as model:
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            theta_value=sess.run(model.theta, feed_dict={model.X: X_train, model.Y: Y_train})
            lr_mse_train =sess.run(model.lr_mse,feed_dict={model.X:X_train,model.Y:Y_train,model.theta_in:theta_value})
            lr_mse_test =sess.run(model.lr_mse,feed_dict={model.X:X_test,model.Y:Y_test,model.theta_in:theta_value})
        
    return theta_value, lr_mse_train,lr_mse_test

In [33]:
theta_value, lr_mse_train, lr_mse_test = run_normal_eq(X_train, Y_train, X_test, Y_test)

__enter__
__exit__


In [34]:
theta_value

array([[1.0026565 ],
       [0.50068945],
       [0.1956349 ],
       [0.09993321]], dtype=float32)

__Normal Equation function with 'with' block of class Linear Regression. Notice: Enter and Exit function doesnt get called here.__

In [35]:
def run_normal_eq2(X_train, Y_train, X_test, Y_test, learning_rate=0.05):
    """
    Implements normal equation using tensorflow, trains the model using training data set
    Tests the model quality by computing mean square error (MSE) of the test data set
    
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    X_test  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_test - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    
    Return a tuple of:
        - np.array of size (k+1 by 1) of regression coefficients
        - mean square error (MSE) of the test data set
        - mean square error (MSE) of the training data set
    """
    # create an instance of the Linear Regression model class  
    n_features = X_train.shape[1]
    #model = LinRegressNormalEq(n_features=n_features, learning_rate=learning_rate)
    
    model=LinRegressNormalEq(n_features=n_features, learning_rate=learning_rate)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        theta_value=sess.run(model.theta, feed_dict={model.X: X_train, model.Y: Y_train})
        lr_mse_train =sess.run(model.lr_mse,feed_dict={model.X:X_train,model.Y:Y_train,model.theta_in:theta_value})
        lr_mse_test =sess.run(model.lr_mse,feed_dict={model.X:X_test,model.Y:Y_test,model.theta_in:theta_value})
        
    return theta_value, lr_mse_train,lr_mse_test

In [38]:
theta_value, lr_mse_train, lr_mse_test = run_normal_eq2(X_train, Y_train, X_test, Y_test)

In [39]:
theta_value

array([[1.0026565 ],
       [0.50068945],
       [0.1956349 ],
       [0.09993321]], dtype=float32)